In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from skimage import filters
from skimage import morphology
import math
from skimage import measure
import time
import sklearn.preprocessing
from sklearn.ensemble import RandomForestClassifier

In [2]:
def img_normalize(sv):
    mx, mn=sv.max(), sv.min()
    sv=(sv-mn)/(mx-mn)
    sv=np.asarray(sv*255, dtype='uint8')
    return sv


dt=[('0' if x<10 else '')+str(x) for x in range(1, 15)]
dy=[[y+'_'+x for y in dt] for x in ['dr', 'h', 'g']]
z=[]
for x in dy:
    z=z+x
print(z)

flag=cv2.IMREAD_GRAYSCALE
#flag=cv2.IMREAD_COLOR
Manuals = [cv2.imread(f"manual1/{x}.tif", flag) for x in z]
Masks= [cv2.imread(f"mask/{x}_mask.tif", flag) for x in z]
Images= [cv2.imread(f"images/{x}.jpg", flag) for x in z]

zupaImagez=[cv2.imread(f"images/{x}.jpg", cv2.IMREAD_COLOR) for x in z]

Creation=list(zip(Manuals, Masks, Images, zupaImagez))

['01_dr', '02_dr', '03_dr', '04_dr', '05_dr', '06_dr', '07_dr', '08_dr', '09_dr', '10_dr', '11_dr', '12_dr', '13_dr', '14_dr', '01_h', '02_h', '03_h', '04_h', '05_h', '06_h', '07_h', '08_h', '09_h', '10_h', '11_h', '12_h', '13_h', '14_h', '01_g', '02_g', '03_g', '04_g', '05_g', '06_g', '07_g', '08_g', '09_g', '10_g', '11_g', '12_g', '13_g', '14_g']


In [10]:
def pad(img, con=3):
    if (len(img.shape)==2):
        dt=np.zeros((img.shape[0]+2*con, img.shape[1]+2*con), dtype='uint8')
        dt[con:dt.shape[0]-con,con:dt.shape[1]-con]=img
    else:
        dt=np.zeros((img.shape[0]+2*con, img.shape[1]+2*con, img.shape[2]), dtype='uint8')
        dt[con:dt.shape[0]-con,con:dt.shape[1]-con]=img
    
    return dt

def getItem(img, x, y, con=3, img2=None):
    #huhu=measure.moments_hu(img[x-con:x+con,y-con:y+con])
    if (img2 is None):
        mcen=measure.moments_central(img[x-con:x+con,y-con:y+con])
        vz=[img[x-con:x+con,y-con:y+con].var()]
    else:
        mcen=measure.moments_central(img2[x-con:x+con,y-con:y+con])
        vz=[img2[x-con:x+con,y-con:y+con,z].var() for z in range(3)]
    col=[img[x,y]]
    lees=np.concatenate((vz, col, np.reshape(mcen, mcen.shape[0]*mcen.shape[1]*mcen.shape[2])))
    return lees

def getImagoData(img, imgalc, mask):
    Cnt=5
    #dissolve=np.zeros((img.shape[0]*img.shape[1], getItem(img, 200, 200).shape[0]))
    dissolve=[]
    img2=pad(img, Cnt)
    img3=pad(imgalc, Cnt)
    
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            #dissolve[i*img.shape[0]+j]=getItem(img2, i+Cnt, j+Cnt, Cnt, img3)
            if (mask[i,j]!=0):
                dissolve.append(getItem(img2, i+Cnt, j+Cnt, Cnt, img3))
    return dissolve


def getImagoRes(bwimg):
    res=np.reshape(bwimg, (bwimg.shape[0]*bwimg.shape[1]))
    return res

def getImagoRandomoRes(resimg, tupls):
    res=[]
    for x in tupls:
        res.append(resimg[x[0], x[1]])
    return np.asarray(res)

#0 - manual, 1 - maska, 2 - oryginał
def randomizer(set_img, Z=40001):
    Cnt=5
    img2=pad(set_img[2], Cnt)
    img3=pad(set_img[3], Cnt)
    dissolve=[]
    tuplez=[]
    setall=set()
    
    for i in range(Z):
        while (True):
            rn=np.random.randint(0, set_img[0].shape[0]*set_img[0].shape[1])
            rny=rn%set_img[1].shape[1]
            rnx=rn//set_img[1].shape[1]
                
            if (set_img[1][rnx,rny]!=0 and
                (set_img[0][rnx,rny]!=0 or
                 np.random.randint(0, 4)%4==0) and
               {(rnx, rny)} not in setall):
                break
        dissolve.append(getItem(img2, rnx+Cnt, rny+Cnt, Cnt, img3))
        setall|={(rnx, rny)}
        tuplez.append((rnx, rny))
        #print(rnx, rny, set_img[0][rnx,rny])
    return np.asarray(dissolve), tuplez

def preproc(data):
    dv=sklearn.preprocessing.normalize(data)
    return dv

In [11]:
t1=time.time()
ln=20
for i in range(ln):
    x1, locs1=randomizer(Creation[i])
    y1=getImagoRandomoRes(Creation[i][0], locs1)
##Normalizacja trainseta, uwypuklenie kluczowych cech
xp1=preproc(x1)

rfc=RandomForestClassifier(n_estimators=50, criterion='gini')
rfc.fit(xp1, y1)
t2=time.time()
print(t2-t1)

158.5491955280304


In [12]:
def evaluator(im1, im2):
    dicc={}
    dicc["1st B"]=len(im1[im1==0])
    dicc["1st W"]=len(im1[im1!=0])
    dicc["2nd B"]=len(im2[im2==0])
    dicc["2nd W"]=len(im2[im2!=0])
    
    dicc["tp"]=len(im2[(im2==im1) & (im2!=0)])
    dicc["tn"]=len(im2[(im2==im1) & (im2==0)])
    dicc["fp"]=len(im2[(im2!=im1) & (im2!=0)])
    dicc["fn"]=len(im2[(im2!=im1) & (im2==0)])
    _=[print(x, dicc[x]) for x in ["tp", "tn", "fp", "fn"]]
    if (dicc["fn"]==0):
        dicc["fn"]=1
    
    dicc["acc"]=(dicc["tp"]+dicc["tn"])/(dicc["tp"]+dicc["tn"]+dicc["fp"]+dicc["fn"])
    dicc["prec"]=dicc["tp"]/(dicc["tp"]+dicc["fp"])
    dicc["rec"]=dicc["tp"]/(dicc["tp"]+dicc["fn"])
    
    return dicc


sgeom=0
for i in range(0, ln):
    t1=time.time()
    
    xt, locst=randomizer(Creation[i])
    yt=getImagoRandomoRes(Creation[i][0], locst)
    xpt=preproc(xt)
    ypt=rfc.predict(xpt)
    dct=evaluator(yt, ypt)
    print(f"Obraz {i+1}:")
    print(f"acc:{dct['acc']}, prec:{dct['prec']}, rec:{dct['rec']}, geom:{math.sqrt(dct['prec']*dct['rec'])}")
    
    t2=time.time()
    print(t2-t1)
    sgeom=sgeom+math.sqrt(dct['prec']*dct['rec'])
print(f"result: {sgeom/ln}")

tp 31261
tn 145353
fp 5242
fn 18145
Obraz 2:
acc:0.8830655846720766, prec:0.8563953647645399, rec:0.6327369145447921, geom:0.7361202080717366
42.91739749908447
tp 18534
tn 148793
fp 3960
fn 28714
Obraz 3:
acc:0.8366308168459158, prec:0.8239530541477728, rec:0.3922705722993566, geom:0.5685178414951896
42.71779179573059
tp 19195
tn 153584
fp 2173
fn 25049
Obraz 4:
acc:0.8638906805465972, prec:0.8983058779483339, rec:0.4338441370581322, geom:0.6242793753062348
45.63193202018738
tp 45588
tn 45691
fp 106142
fn 2580
Obraz 5:
acc:0.4563927180364098, prec:0.3004547551571871, rec:0.9464374688589935, geom:0.5332556966199393
44.07840418815613
tp 51075
tn 54912
fp 88858
fn 5156
Obraz 6:
acc:0.5299323503382483, prec:0.36499610527895493, rec:0.9083068058544219, geom:0.5757850697397701
44.18585205078125
tp 25470
tn 137899
fp 4650
fn 31982
Obraz 7:
acc:0.8168409157954211, prec:0.8456175298804781, rec:0.44332660307735156, geom:0.6122783248038209
44.78805232048035
tp 14908
tn 139679
fp 3972
fn 41442
Obr